In [ ]:
import matplotlib.pyplot as plt
import MDAnalysis as mda
import numpy as np
from matplotlib.markers import MarkerStyle as markerstyle
import math
from scipy.interpolate import griddata
import pandas
from scipy import constants
import matplotlib.patches as patches
import matplotlib.cm as cm

### Read Input files

In [ ]:
#Energies_File
ener_file = "ADD INPUT FILE HERE"   #Reweighted Energy file, e.g. "2D_Martini3/Filtered_data_Energy.dat"
dist_file = "ADD MASTER DATA FILE HERE"   #Distance file, e.g. "2D_Martini3/Final_data.txt"

ener_data = pandas.read_csv(ener_file,comment='#',sep='\t',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file


#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)

#Merge two dataframes
master_data = dist_data.join(ener_data.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')

In [ ]:
#Remove instances where protein came off membrane 
timesteps_array = master_data['Timestep']/1e6

#Timepoints to ignore 
 #33.0 - 66.0 us 
#95.2 - 97.8 us
#101.95 to 103.3 us 
#127.6 to 128.5 us

mask1 = (timesteps_array > 33.0) & (timesteps_array < 66.0)
mask2 = (timesteps_array > 95.2) & (timesteps_array < 97.8)
mask3 = (timesteps_array > 101.95) & (timesteps_array < 103.3)
mask4 = (timesteps_array > 127.6) & (timesteps_array < 128.5)
mask5 = (timesteps_array > 139) & (timesteps_array < 142.5)


mask = mask1 | mask2 | mask3 | mask4 | mask5

master_data = master_data[~mask]

In [ ]:
bound_mask = (master_data['d1_mda'] < 13.0) & (master_data['d2_mda'] < 13.0)
# bound_mask = (master_data['RMSD-BtoA'] < 2.0)
unbound_mask = ((master_data['d1_mda'] > 13.0) | (master_data['d2_mda'] > 13.0)) & ((master_data['d1_mda'] < 19.0) & (master_data['d2_mda'] < 19.0))

bd_data = master_data[bound_mask]
unb_data = master_data[unbound_mask]

## Reweighting

Set rwt_bool = True to turn on reweighting

In [ ]:
rwt_bool = True
if rwt_bool:
    bd_rwt = bd_data['rbias']
    unb_rwt = unb_data['rbias']
else:
    bd_rwt = np.ones(len(bd_data))
    unb_rwt = np.ones(len(unb_data))

## Calculating Ensemble Averages


In [ ]:
"""
2D
"""
"""
------- Defining Constants ------------
"""
k = constants.value('Boltzmann constant')
Ava_no = constants.value('Avogadro constant')
temp = 310
kbt = (k*temp*Ava_no)

#Add new column to data frame called 'State'

col_names = ['LJ-Pro-Pro','LJ-Pro-W','Col-Pro-Pro','Col-Pro-W','Col-Pro-Mem','LJ-Pro-Mem','LJ-W-W','Col-W-W','LJ-Mem-Mem','Col-Mem-Mem','LJ-Mem-W','Col-Mem-W']

rwt_bool = True

bound_mask = (master_data['d1_mda'] < 12.0) & (master_data['d2_mda'] < 12.0) #& (master_data['Timestep'] < 90000000)
cry_mask = (master_data['RMSD-BtoA'] < 2.0)
unbound_mask = ((master_data['d1_mda'] > 12.0) | (master_data['d2_mda'] > 12.0)) & ((master_data['d1_mda'] < 19.0) & (master_data['d2_mda'] < 19.0)) #& (master_data['Timestep'] < 90000000)

bd_energies = {}
unb_energies = {}
cry_energies = {}

std_bd = {}
std_unb = {}
std_cry = {}
for col in col_names:
    if rwt_bool:
        bd_energies[col] = np.average(master_data[col][bound_mask],weights=np.exp(master_data['rbias'][bound_mask]/kbt),axis=0)
        unb_energies[col] = np.average(master_data[col][unbound_mask],weights=np.exp(master_data['rbias'][unbound_mask]/kbt),axis=0)
        cry_energies[col] = np.average(master_data[col][cry_mask],weights=np.exp(master_data['rbias'][cry_mask]/kbt),axis=0)

        std_bd[col] = np.sqrt(np.average((master_data[col][bound_mask]-bd_energies[col])**2,weights=np.exp(master_data['rbias'][bound_mask]/kbt)))
        std_unb[col] = np.sqrt(np.average((master_data[col][unbound_mask]-unb_energies[col])**2,weights=np.exp(master_data['rbias'][unbound_mask]/kbt)))
        std_cry[col] = np.sqrt(np.average((master_data[col][cry_mask]-cry_energies[col])**2,weights=np.exp(master_data['rbias'][cry_mask]/kbt)))
    else:
        bd_energies[col] = np.average(master_data[col][bound_mask])
        unb_energies[col] = np.average(master_data[col][unbound_mask])
        cry_energies[col] = np.average(master_data[col][cry_mask])

        std_bd[col] = np.std(master_data[col][bound_mask])
        std_unb[col] = np.std(master_data[col][unbound_mask])
        std_cry[col] = np.std(master_data[col][cry_mask])
        

bonded_terms = ['Bond','HP','G96','PDih','ImPDih']
bonded_sum_bd = np.average(master_data[bonded_terms][bound_mask].sum(axis=1),weights=np.exp(master_data['rbias'][bound_mask]/kbt))
bonded_sum_unbd = np.average(master_data[bonded_terms][unbound_mask].sum(axis=1),weights=np.exp(master_data['rbias'][unbound_mask]/kbt))
bonded_sum_cry = np.average(master_data[bonded_terms][cry_mask].sum(axis=1),weights=np.exp(master_data['rbias'][cry_mask]/kbt))

std_bonded_bd = np.sqrt(np.average((master_data[bonded_terms][bound_mask].sum(axis=1)-bonded_sum_bd)**2,weights=np.exp(master_data['rbias'][bound_mask]/kbt)))
std_bonded_unb = np.sqrt(np.average((master_data[bonded_terms][unbound_mask].sum(axis=1)-bonded_sum_unbd)**2,weights=np.exp(master_data['rbias'][unbound_mask]/kbt)))
std_bonded_cry = np.sqrt(np.average((master_data[bonded_terms][cry_mask].sum(axis=1)-bonded_sum_cry)**2,weights=np.exp(master_data['rbias'][cry_mask]/kbt)))
                        

col_names.append('Bonded')
bd_energies['Bonded'] = bonded_sum_bd
unb_energies['Bonded'] = bonded_sum_unbd
cry_energies['Bonded'] = bonded_sum_cry

std_bd['Bonded']=std_bonded_bd
std_unb['Bonded']=std_bonded_unb
std_cry['Bonded']=std_bonded_cry



## Plotting parameters

In [ ]:
f_size=18
leg_size=16
tick_size=17
fig_size= (6,4)

In [ ]:
fig,ax = plt.subplots(figsize=fig_size)

ax.plot(bd_data['d1_mda'],bd_data['d2_mda'],label='Bound',linewidth=1.2,linestyle='',color='steelblue',marker='o',markersize=5)
ax.plot(unb_data['d1_mda'],unb_data['d2_mda'],label='Unbound',linewidth=1.2,linestyle='',color='gold',marker='o',markersize=5)

ax.set_xlabel('d1 (nm)',fontsize=f_size)
ax.set_ylabel('d2 (nm)',fontsize=f_size)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
plt.show()

In [ ]:
subset_ener = ['Bond','HP','G96','PDih','ImPDih','Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-Mem','LJ-Pro-Mem','Col-Pro-W','LJ-Pro-W','Col-Mem-Mem','LJ-Mem-Mem','Col-Mem-W','LJ-Mem-W','Col-W-W','LJ-W-W']
bound_energy = bd_data[subset_ener].sum(axis=1)
unbound_energy = unb_data[subset_ener].sum(axis=1)

bound_dict = {}
unbound_dict = {}

fig,ax = plt.subplots(figsize=(6,4))
bonded_terms = ['Bond','HP','G96','PDih','ImPDih']
bonded_sum_bd = bd_data[bonded_terms].sum(axis=1)
bonded_sum_unbd = unb_data[bonded_terms].sum(axis=1)

bound_dict['Bonded'] = [np.mean(bonded_sum_bd), np.std(bonded_sum_bd)]
unbound_dict['Bonded'] = [np.mean(bonded_sum_unbd),np.std(bonded_sum_unbd)]

ax.hist(bonded_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(bonded_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Bonded Interactions',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
fig.tight_layout()
plt.savefig('EnergyPlots/2D_Bonded.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-Pro']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col. (Pro-Pro) Interactions',fontsize=f_size)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-W']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col. (Pro-W) Interactions',fontsize=f_size)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-Mem']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col. (Pro-Mem) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_Col-ProMem.svg',format='svg',dpi=600,transparent=True)
plt.show()


In [ ]:
def plot_vs_cutoff(plot_data,ener_col,cutoffs,ax,colors=None):
    n_configs=[]
    prev_cutoff = 0.0
    for cutoff in cutoffs:
        mask = (plot_data['RMSD-BtoA'] < cutoff) & (plot_data['RMSD-BtoA'] >= prev_cutoff)
        data = plot_data[mask]
        energy = data[[ener_col]].sum(axis=1)
        if colors is not None:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True,color=colors[cutoff])
        else:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True)

        n_configs.append(len(energy))
        prev_cutoff = cutoff

    return n_configs



        


In [ ]:
col_names = 'LJ-Pro-Pro','Col-Pro-Pro','LJ-Pro-Mem','Col-Pro-Mem'

fig,ax=plt.subplots(1,4,figsize=(15,4))

t_size= 18
lw = 3
f_size= 16
for i in range(len(col_names)):
    row_idx = i//2
    col_idx = i%2
    # nconfigs = plot_vs_cutoff(master_data,col_names[i],cutoffs=[0.2,0.4,0.6,0.8,1.0,1.4,1.8,2.0],ax=ax[i])
    nconfigs = plot_vs_cutoff(master_data,col_names[i],cutoffs=[0.6,2.0],ax=ax[i])
    ax[i].set_title(col_names[i],fontsize=f_size+6)

    print(f'{col_names[i]}: {nconfigs}')


# ax[3].legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2,bbox_to_anchor=(1.5,1))

# ax[1].set_xticks([-12500,-12000,-11500,-11000])
ax[0].set_xlim(left=-13000,right=-10500)
ax[1].set_xlim(left=-1450,right=-1100)
ax[2].set_xlim(left=-17500,right=-13000)plt.savefig('EnergyPlots/2D_Col-ProMem.svg',format='svg',dpi=600,transparent=True)
ax[3].set_xlim(left=-900,right=-200)
for ax_ in ax.flat:
    # ax_.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
    # ax_.set_ylabel('Frequency',fontsize=f_size)
    
    ax_.tick_params(axis='both',labelsize=t_size)
    ax_.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
    

fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-Pro']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.set_title('LJ (Pro-Pro) Interactions',fontsize=f_size)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-W']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.set_title('LJ (Pro-W) Interactions',fontsize=f_size)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-Mem']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.set_title('LJ (Pro-Mem) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_LJ-ProMem.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
ig,ax = plt.subplots(figsize=fig_size)

lj_terms='LJ-W-W'
ljW_bd= bd_data[lj_terms]
ljW_unbd = unb_data[lj_terms]
bound_dict[lj_terms] = [np.mean(ljW_bd), np.std(ljW_bd)]
unbound_dict[lj_terms] = [np.mean(ljW_unbd),np.std(ljW_unbd)]

ax.hist(ljW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(ljW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_title('LJ Energy (W-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_LJ-WW.svg',format='svg',dpi=600,transparent=True)

fig,ax = plt.subplots(figsize=fig_size)
lj_terms='LJ-Mem-W'
ljW_bd= bd_data[lj_terms]
ljW_unbd = unb_data[lj_terms]
bound_dict[lj_terms] = [np.mean(ljW_bd), np.std(ljW_bd)]
unbound_dict[lj_terms] = [np.mean(ljW_unbd),np.std(ljW_unbd)]

ax.hist(ljW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(ljW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_title('LJ Energy (Mem-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_LJ-MemW.svg',format='svg',dpi=600,transparent=True)

fig,ax = plt.subplots(figsize=fig_size)
lj_terms='LJ-Mem-Mem'
ljW_bd= bd_data[lj_terms]
ljW_unbd = unb_data[lj_terms]
bound_dict[lj_terms] = [np.mean(ljW_bd), np.std(ljW_bd)]
unbound_dict[lj_terms] = [np.mean(ljW_unbd),np.std(ljW_unbd)]
ax.hist(ljW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(ljW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_title('LJ (Mem-Mem) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_LJ-MemMem.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
col_terms = 'Col-W-W'
colW_bd= bd_data[col_terms]
colW_unbd = unb_data[col_terms]
bound_dict[col_terms] = [np.mean(colW_bd), np.std(colW_bd)]
unbound_dict[col_terms] = [np.mean(colW_unbd),np.std(colW_unbd)]

ax.hist(colW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_title('Col. Energy (W-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_Col-WW.svg',format='svg',dpi=600,transparent=True)

fig,ax = plt.subplots(figsize=(6,4))
col_terms = 'Col-Mem-W'
colW_bd= bd_data[col_terms]
colW_unbd = unb_data[col_terms]
bound_dict[col_terms] = [np.mean(colW_bd), np.std(colW_bd)]
unbound_dict[col_terms] = [np.mean(colW_unbd),np.std(colW_unbd)]

ax.hist(colW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_title('Col. Energy (Mem-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_Col-MemW.svg',format='svg',dpi=600,transparent=True)

fig,ax = plt.subplots(figsize=(6,4))
col_terms = 'Col-Mem-Mem'

colW_bd= bd_data[col_terms]
colW_unbd = unb_data[col_terms]
bound_dict[col_terms] = [np.mean(colW_bd), np.std(colW_bd)]
unbound_dict[col_terms] = [np.mean(colW_unbd),np.std(colW_unbd)]

ax.hist(colW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_title('Col. Energy (Mem-Mem) Interactions',fontsize=f_size-2)
fig.tight_layout()
plt.savefig('EnergyPlots/2D_Col-MemMem.svg',format='svg',dpi=600,transparent=True)

plt.show()


In [ ]:
## FIlter configs for contact analysis

mask1 = master_data['RMSD-BtoA'] <= 0.6
mask2 = (master_data['RMSD-BtoA'] > 0.6) & (master_data['RMSD-BtoA'] <= 2.0)

rmsd1_times = master_data['Timestep'][mask1].to_numpy()
rmsd2_times = master_data['Timestep'][mask2].to_numpy()

In [ ]:
print(rmsd1_times)
print(rmsd2_times)

In [ ]:
time_dict={}
for i in range(len(rmsd1_times)):
    time_dict[rmsd1_times[i]] = 1

for i in range(len(rmsd2_times)):
    time_dict[rmsd2_times[i]] = 2


In [ ]:
with open("rmsd_times_2D.py",'w') as fl1:
    fl1.write("time_dict = {}")
    fl1.write("\n")
    for time,st in time_dict.items():
        fl1.write("time_dict[{:.1f}] = {}".format(time,st))
        fl1.write("\n")